##Rotten Tomatoes review classification using Keras without pre-trained embedding layer

In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install wordvecpy
from wordvecpy import TextProcessor, FastVectokenizer
!pip install pymagnitude
import pymagnitude
import timeit

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
  Stored in directory: /root/.cache/pip/wheels/0a/37/8d/929b022daf780d0597ee8aa6eac33e9b69cd4b09215d1944a1
Successfully built wordvecpy
     |████████████████████████████████| 5.4MB 40.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [7]:
import spacy
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


Upload our Rotten Tomatoes reviews and the 200-dimensional twitter pre-trained word embedding .magnitude file for use with pymagnitude.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we load our dataset into a dataframe and preprocess the review text

In [0]:
df = pd.read_csv("/content/drive/My Drive/rotten_tomatoes_reviews.csv")
vectors = pymagnitude.Magnitude("/content/drive/My Drive/glove.6B.200d.magnitude")

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.Review, df.Freshness, test_size = 0.2, random_state = 0)

x_train_processed = TextProcessor(x_train, lemmatizer='spaCy en')
x_test_processed = TextProcessor(x_test, lemmatizer='spaCy en')

Now we create generate the integer embeddings and vector dictionary to use as input to, and the weights of, our Keras embedding layer

In [11]:
vectokenizer = FastVectokenizer(x_train_processed.transform(), vectors, x_test_processed.transform())
x_train, x_test, vector_dict = vectokenizer.to_keras()
max_sentence_length = vectokenizer.max_sentence_length
vocab_size = vectokenizer.max_words

Using TensorFlow backend.


The first model we'll make will be a simple CNN classifier



In [12]:
from keras.models import Input, Model
from keras.layers import Conv1D, AveragePooling1D, Dropout, SpatialDropout1D
from keras.layers import BatchNormalization, MaxPooling1D, Dense, Flatten
from keras.layers import Embedding
from keras.initializers import Constant

model_input = Input(shape = (max_sentence_length,))
layers_cnn = Embedding(vocab_size, 200, embeddings_initializer=Constant(vector_dict), 
                       input_length=max_sentence_length, trainable=False)(model_input)
layers_cnn = Conv1D(filters = 128, kernel_size = 7, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Conv1D(filters = 128, kernel_size = 5, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Conv1D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Conv1D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Conv1D(filters = 128, kernel_size = 2, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Conv1D(filters = 128, kernel_size = 1, strides = 1, activation = 'relu')(layers_cnn)
layers_cnn = BatchNormalization()(layers_cnn)
layers_cnn = Flatten()(layers_cnn)
cnn_model_output = Dense(1, activation = 'sigmoid')(layers_cnn)

cnn_model = Model(inputs = model_input, outputs = cnn_model_output)


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
cnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
t0=timeit.default_timer()
cnn_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
384000/384000 [==============================] - 58s 150us/step - loss: 0.5299 - acc: 0.7331
Epoch 2/20
384000/384000 [==============================] - 45s 117us/step - loss: 0.4497 - acc: 0.7896
Epoch 3/20
384000/384000 [==============================] - 45s 116us/step - loss: 0.3909 - acc: 0.8242
Epoch 4/20
384000/384000 [==============================] - 45s 116us/step - loss: 0.3359 - acc: 0.8551
Epoch 5/20
384000/384000 [==============================] - 45s 116us/step - loss: 0.2868 - acc: 0.8804
Epoch 6/20
384000/384000 [==============================] - 44s 116us/step - loss: 0.2436 - acc: 0.9006
Epoch 7/20
384000/384000 [==============================] - 44s 114us/step - loss: 0.2071 - acc: 0.9167
Epoch 8/20
384000/384000 [==============================] - 44s 114us/step - loss: 0.1787 - acc: 0.9292
Epoch 9/20
384000/384000 [==============================] - 45s 117us/step - loss: 0.1555 - acc: 0.9390
Epoch 10/20
3840

In [15]:
score = cnn_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

96000/96000 [==============================] - 9s 96us/step

loss is: 0.7529
accuracy is: 0.84875


Not bad.  Now to try a simple RNN.

In [16]:
from keras.layers import Bidirectional, CuDNNLSTM

layers_rnn = Embedding(vocab_size, 300)(model_input)
layers_rnn = Bidirectional(CuDNNLSTM(128))(layers_rnn)
layers_rnn = BatchNormalization()(layers_rnn)
layers_rnn = Dropout(.35)(layers_rnn)
rnn_model_output = Dense(1, activation = 'sigmoid')(layers_rnn)

rnn_model = Model(inputs = model_input, outputs = rnn_model_output)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
rnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
t0=timeit.default_timer()
rnn_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

Epoch 1/20
384000/384000 [==============================] - 86s 224us/step - loss: 0.4318 - acc: 0.7977
Epoch 2/20
384000/384000 [==============================] - 85s 221us/step - loss: 0.3075 - acc: 0.8689
Epoch 3/20
384000/384000 [==============================] - 85s 221us/step - loss: 0.2237 - acc: 0.9090
Epoch 4/20
384000/384000 [==============================] - 84s 220us/step - loss: 0.1551 - acc: 0.9395
Epoch 5/20
384000/384000 [==============================] - 83s 217us/step - loss: 0.1084 - acc: 0.9590
Epoch 6/20
384000/384000 [==============================] - 82s 215us/step - loss: 0.0778 - acc: 0.9714
Epoch 7/20
384000/384000 [==============================] - 83s 215us/step - loss: 0.0587 - acc: 0.9785
Epoch 8/20
384000/384000 [==============================] - 82s 214us/step - loss: 0.0454 - acc: 0.9837
Epoch 9/20
384000/384000 [==============================] - 82s 215us/step - loss: 0.0367 - acc: 0.9870
Epoch 10/20
384000/384000 [==============================] - 82s

In [19]:
score = cnn_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

96000/96000 [==============================] - 9s 94us/step

loss is: 0.7529
accuracy is: 0.84875


**Eh**.  Not good.

In [0]:
from keras.layers import concatenate
comb_para_model = concatenate([layers_cnn, layers_rnn])
comb_model_layer = Dense(256, activation = 'selu')(comb_para_model)
comb_model_output = Dense(1, activation = 'sigmoid')(comb_model_layer)

comb_model = Model(inputs = model_input, outputs = comb_model_output)

In [0]:
comb_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
t0=timeit.default_timer()
comb_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

Epoch 1/20
384000/384000 [==============================] - 120s 313us/step - loss: 0.0122 - acc: 0.9958
Epoch 2/20
384000/384000 [==============================] - 118s 306us/step - loss: 0.0143 - acc: 0.9950
Epoch 3/20
384000/384000 [==============================] - 117s 306us/step - loss: 0.0132 - acc: 0.9955
Epoch 4/20
384000/384000 [==============================] - 118s 306us/step - loss: 0.0124 - acc: 0.9959
Epoch 5/20
384000/384000 [==============================] - 118s 306us/step - loss: 0.0115 - acc: 0.9961
Epoch 6/20
384000/384000 [==============================] - 118s 308us/step - loss: 0.0116 - acc: 0.9961
Epoch 7/20
384000/384000 [==============================] - 118s 308us/step - loss: 0.0112 - acc: 0.9962
Epoch 8/20
384000/384000 [==============================] - 118s 309us/step - loss: 0.0110 - acc: 0.9964
Epoch 9/20
384000/384000 [==============================] - 118s 308us/step - loss: 0.0109 - acc: 0.9964
Epoch 10/20
384000/384000 [============================

In [23]:
score = comb_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

96000/96000 [==============================] - 19s 193us/step

loss is: 0.84
accuracy is: 0.8801041666666667


Well, 88% is the best we've gotten out of the NNs so far.  Maybe it's time to train it for another 10 epochs and see if it improves

In [24]:
t0=timeit.default_timer()
comb_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

score = comb_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

Epoch 1/20
384000/384000 [==============================] - 117s 304us/step - loss: 0.0080 - acc: 0.9973
Epoch 2/20
384000/384000 [==============================] - 117s 305us/step - loss: 0.0089 - acc: 0.9970
Epoch 3/20
384000/384000 [==============================] - 117s 304us/step - loss: 0.0078 - acc: 0.9973
Epoch 4/20
384000/384000 [==============================] - 117s 304us/step - loss: 0.0082 - acc: 0.9971
Epoch 5/20
384000/384000 [==============================] - 117s 304us/step - loss: 0.0080 - acc: 0.9973
Epoch 6/20
384000/384000 [==============================] - 117s 303us/step - loss: 0.0082 - acc: 0.9973
Epoch 7/20
384000/384000 [==============================] - 116s 303us/step - loss: 0.0080 - acc: 0.9973
Epoch 8/20
384000/384000 [==============================] - 117s 303us/step - loss: 0.0076 - acc: 0.9974
Epoch 9/20
384000/384000 [==============================] - 116s 303us/step - loss: 0.0079 - acc: 0.9973
Epoch 10/20
384000/384000 [============================

About the same as before.  Let's try one more


In [0]:
layers_crnn = Embedding(vocab_size, 300)(model_input)
layers_crnn = Bidirectional(CuDNNLSTM(128, return_sequences=True))(layers_crnn)
layers_crnn = SpatialDropout1D(.2)(layers_crnn)
layers_crnn = Conv1D(filters = 128, kernel_size = 7, strides = 1, activation = 'relu')(layers_crnn)
layers_crnn = BatchNormalization()(layers_crnn)
layers_crnn = Conv1D(filters = 128, kernel_size = 5, strides = 1, activation = 'relu')(layers_crnn)
layers_crnn = BatchNormalization()(layers_crnn)
layers_crnn = Conv1D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu')(layers_crnn)
layers_crnn = BatchNormalization()(layers_crnn)
layers_crnn = Conv1D(filters = 128, kernel_size = 2, strides = 1, activation = 'relu')(layers_crnn)
layers_crnn = BatchNormalization()(layers_crnn)
layers_crnn = CuDNNLSTM(128)(layers_crnn)
layers_crnn = Dropout(.2)(layers_crnn)
layers_crnn = Dense(128, activation = 'relu')(layers_crnn)
crnn_model_output = Dense(1, activation = 'sigmoid')(layers_crnn)

crnn_model = Model(inputs = model_input, outputs = crnn_model_output)

In [0]:
crnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [27]:
t0=timeit.default_timer()
crnn_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

Epoch 1/20
384000/384000 [==============================] - 119s 310us/step - loss: 0.4409 - acc: 0.7903
Epoch 2/20
384000/384000 [==============================] - 116s 302us/step - loss: 0.3097 - acc: 0.8691
Epoch 3/20
384000/384000 [==============================] - 116s 303us/step - loss: 0.2243 - acc: 0.9107
Epoch 4/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.1572 - acc: 0.9402
Epoch 5/20
384000/384000 [==============================] - 116s 302us/step - loss: 0.1104 - acc: 0.9594
Epoch 6/20
384000/384000 [==============================] - 116s 302us/step - loss: 0.0802 - acc: 0.9712
Epoch 7/20
384000/384000 [==============================] - 116s 303us/step - loss: 0.0606 - acc: 0.9789
Epoch 8/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.0472 - acc: 0.9840
Epoch 9/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.0376 - acc: 0.9875
Epoch 10/20
384000/384000 [============================

In [28]:
score = crnn_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

96000/96000 [==============================] - 20s 205us/step

loss is: 0.7913
accuracy is: 0.8748125


So, still not coming close to 90%

In [0]:
layers_mini = Embedding(vocab_size, 300)(model_input)
layers_mini = Bidirectional(CuDNNLSTM(64, return_sequences = True))(layers_mini)
layers_mini = Conv1D(filters = 64, kernel_size = 7, strides = 1, activation = 'relu')(layers_mini)
layers_mini = MaxPooling1D(3, strides=1)(layers_mini)
layers_mini = Bidirectional(CuDNNLSTM(64, return_sequences = True))(layers_mini)
layers_mini = Conv1D(filters = 64, kernel_size = 5, strides = 1, activation = 'relu')(layers_mini)
layers_mini = MaxPooling1D(3, strides=1)(layers_mini)
layers_mini = Bidirectional(CuDNNLSTM(64, return_sequences = False))(layers_mini)
layers_mini = BatchNormalization()(layers_mini)
layers_mini = Dropout(.25)(layers_mini)
layers_mini = Dense(256, activation = 'relu')(layers_mini)
layers_mini = BatchNormalization()(layers_mini)
layers_mini = Dropout(.25)(layers_mini)
layers_mini = Dense(256, activation = 'relu')(layers_mini)
mini_model_output = Dense(1, activation = 'sigmoid')(layers_mini)

mini_model = Model(inputs = model_input, outputs = mini_model_output)

mini_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [30]:
t0=timeit.default_timer()
mini_model.fit(x_train, y_train, epochs = 20, batch_size = 128)
t1=timeit.default_timer()
print('\nTime to train: {}\n'.format(round(t1-t0, 3)))

Epoch 1/20
384000/384000 [==============================] - 122s 316us/step - loss: 0.4452 - acc: 0.7913
Epoch 2/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.3090 - acc: 0.8698
Epoch 3/20
384000/384000 [==============================] - 115s 300us/step - loss: 0.2164 - acc: 0.9144
Epoch 4/20
384000/384000 [==============================] - 115s 301us/step - loss: 0.1472 - acc: 0.9443
Epoch 5/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.1021 - acc: 0.9624
Epoch 6/20
384000/384000 [==============================] - 116s 302us/step - loss: 0.0727 - acc: 0.9740
Epoch 7/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.0536 - acc: 0.9809
Epoch 8/20
384000/384000 [==============================] - 116s 301us/step - loss: 0.0421 - acc: 0.9850
Epoch 9/20
384000/384000 [==============================] - 116s 303us/step - loss: 0.0344 - acc: 0.9882
Epoch 10/20
384000/384000 [============================

In [31]:
score = mini_model.evaluate(x_test, y_test)

print('\nloss is: ' + str(score[0].round(4)))
print('accuracy is: ' + str(score[1]))

96000/96000 [==============================] - 27s 283us/step

loss is: 1.0435
accuracy is: 0.8735


Total failure.  Worse than before.